In [60]:
import cryo
import polars as pl
import binascii
import web3
import json
from eth_abi import decode
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

In [61]:
# Functions
def bytes_to_hexstr(b: any) -> str:
    if isinstance(b,list):
        return [bytes_to_hexstr(a) for a in b]
    return '0x' + b.hex()

In [62]:
# Function Signatures 4 bytes
getBlocknumber_4b = '42cbb15c'

In [90]:
txs = cryo.collect('transactions', blocks=['latest'], rpc ='https://eth.merkle.io')

In [91]:
txs.head()

block_number,transaction_index,transaction_hash,nonce,from_address,to_address,value_binary,value_string,value_f64,input,gas_limit,gas_used,gas_price,transaction_type,max_priority_fee_per_gas,max_fee_per_gas,success,n_input_bytes,n_input_zero_bytes,n_input_nonzero_bytes,chain_id
u32,u64,binary,u64,binary,binary,binary,str,f64,binary,u64,u64,u64,u32,u64,u64,bool,u32,u32,u32,u64
21945502,0,"b""h\xab\xb4\xc9rj\xf0\xa4\x18\x81]\xb9+Z{\x9d\x8d\x0e\xd5""\xfa~B\xc34\xae\xa5\xa2\xa8\x15\x8d\xb9""",784,"b""\x0cRA~\xa6~\xfe\xd1H\x1de\x20\xaaD(A\xafZ\xd9\x8e""","b""\x80\xa6Lm\x7f\x12\xc4{|f\xc5\xb4\xe2\x0er\xbc\x1f\xcd]\x9e""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb1\xa2\xbc.\xc5\x00\x00""","""50000000000000000""",5.0000e16,"b""\x08\x88\x90\xdc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00uT\x9f\xb5<\xe6\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",333015,195296,17391286973,2,15000000000,18042696454,true,260,164,96,1
21945502,1,"b"")M.^C\xee\x9f\x83\xeb\x8b\xed\xdb\xf7\x00'\xadK\x03Fa\xd8\xb7\xae\xc3\xf1Q\xe2\x0c\xef1\x83]""",4270735,"b""\xae/\xc4\x83R{\x8e\xf9\x9e\xb5\xd9\xb4Hu\xf0\x05\xba\x1f\xae\x13""","b""\x1f/\x10\xd1\xc4\x07w\xae\x1d\xa7BE\e\x82\x8f\xf3m\xf3\x87""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xef""","""239""",239.0,"b""+\xa6\\x95\xd4\xb1\xc32\x1c\xf8\x98\xd2YI\xf4\x1dP\xbe-\xb5\xbc\x1d[7zB\x20@\xd1o\xc0$j\xd3\x16\x0c\xcc\x09\xb8\xd0\xd3\xa2\xcd(\xael/\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.""…",783804,548663,5186251918,2,5186251917,5186251918,true,639,133,506,1
21945502,2,"b""e\x83\xb8%\xfd\xba7E>T\x0bI\xa6\xc0\xc8\x18\x80\x13\xf0\xe4\xec\xb6\x94\xdc?D\x9b\xb9\x07\xfd\x0cM""",46,"b""\x00H\xb6\x036+\xa20i*\xfe\xe6=\xee\x15#\xe1[Oq""","b""\x80\xa6Lm\x7f\x12\xc4{|f\xc5\xb4\xe2\x0er\xbc\x1f\xcd]\x9e""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""","""0""",0.0,"b""=\x0e>\xc5\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00)%\xf5\xd2\xef7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",552482,245757,3991286973,2,1600000000,4642696454,true,292,190,102,1
21945502,3,"b""\xdd\xc8\x9b\x06\xc8Q#""\xfdo\x91_\x80\x16\xcf\x10\xe1\x94\xbd\xcb\x01\xb9<\x82k\x1fXU\x0a\xfc\x11\xef""",303,"b""\xce\xc4\x97\x80\x01\x03<\xe3z\x1e'\xc6\xa0\xd8\xaa\x14\x9a\x85Gw""","b""\xbc\xf5\x9e\xe0\xb7C\x12\xa3\xfaJ\x9f\xd2X\xad\xca\x0a\x1ca\xa5\xaa""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""","""0""",0.0,"b""\x09^\xa7\xb3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\xa6Lm\x7f\x12\xc4{|f\xc5\xb4\xe2\x0er\xbc\x1f\xcd]\x9e\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff""…",56352,46577,5391286973,2,3000000000,6432828194,true,68,12,56,1
21945502,4,"b""3)\x1e\x84Mi\x04\xde\x9a\xf8\x1b{\xf4\x07\x0dSs4\x11H\x07Z\\xe5\x15\xe4\x02\xce!\xfbo\x97""",227449,"b""\xfc\x99(\xf6Y\x0d\x857R\x82K\x0b@:j\xe3g\x85\xe55""","b""\x00\x00\x00\x00\x00\x9eP\xa7\xdd\xb7\xa7\xb0\xe2\xeef\x04\xfd\x12\x0eI""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""","""0""",0.0,"b""\x9e!j\x04\x85\x05\x19\x08\x11\xb8\x15\xef\xb8\xf5\x81\x19J\xe7\x90\x06\xd2N\x0d\x81Kv\x97\xf6,\xde\x00\x00\x00\x00\x00\x00|\xc2\xe5^\x00\x00\x00\x00\x04?\xa3\x00D\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",930415,651291,6715096218,2,4323809245,7432482309,true,713,388,325,1


## Balance

In [92]:
# ERC20s
USDT_ADDR ="0xdac17f958d2ee523a2206206994597c13d831ec7"
USDC_ADDR="0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48" 
# WALLET = Binance 18
RANDOM_WALLET="0xf70da97812CB96acDF810712Aa562db8dfA3dbEF" 
# block 
BLOCKS="21922375"


In [93]:
usdc_balance = cryo.collect('erc20_balances', blocks=[BLOCKS], contract=['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], address=['0xf70da97812CB96acDF810712Aa562db8dfA3dbEF'], rpc='https://eth.merkle.io')

In [94]:
usdc_balance

block_number,erc20,address,balance_binary,balance_string,balance_f64,chain_id
u32,binary,binary,binary,str,f64,u64
21945504,"b""\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.\x9e\xb0\xce6\x06\xebH""","b""\xf7\x0d\xa9x\x12\xcb\x96\xac\xdf\x81\x07\x12\xaaV-\xb8\xdf\xa3\xdb\xef""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00:\x86\xfa\xc9\x8d""","""251372685709""",2.5137e11,1


In [95]:
output_addr=usdc_balance['erc20'][0]

In [96]:
output_addr.hex()

'a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

In [97]:
usdc_balance['address'][0].hex()

'f70da97812cb96acdf810712aa562db8dfa3dbef'

In [98]:
usdc_balance['balance_binary'][0]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00:\x86\xfa\xc9\x8d'

In [99]:
balance= usdc_balance['balance_binary'][0]

In [101]:
int((balance).hex(),16)

251372685709

In [102]:
usdc_balance['balance_string'][0]

'251372685709'

In [103]:
balance_raw = decode(['uint256'], usdc_balance['balance_binary'][0])

In [104]:
balance_raw

(251372685709,)

In [105]:
 balance_raw[0] / 10**6

251372.685709

In [106]:
# BLOCKS="21922375" >> usdc 472,885.333888

## Aave Pools Data

In [107]:
#  AAVE
# https://etherscan.io/address/0x87870bca3f3fd6335c3f4ce8392d69350b4fa4e2#readProxyContract

In [296]:
MULTICALL3_ADDRESS = '0xcA11bde05977b3631167028862bE2a173976CA11'
MULTICALL3_ABI=json.loads('[{"inputs":[{"internalType":"bool","name":"requireSuccess","type":"bool"},{"components":[{"internalType":"address","name":"target","type":"address"},{"internalType":"bytes","name":"callData","type":"bytes"}],"internalType":"struct Multicall3.Call[]","name":"calls","type":"tuple[]"}],"name":"tryAggregate","outputs":[{"components":[{"internalType":"bool","name":"success","type":"bool"},{"internalType":"bytes","name":"returnData","type":"bytes"}],"internalType":"struct Multicall3.Result[]","name":"returnData","type":"tuple[]"}],"stateMutability":"payable","type":"function"}]')

In [297]:
AAVE_POOL_PROVIDER_ADDRESS = "0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e"
AAVE_POOL_DATA_PROVIDER_ADDRESS = "0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3"
RATE_STRATEGY_V2= "0x9ec6F08190DeA04A54f8Afc53Db96134e5E3FdFB"
DAI_ADDRESS = "0x6b175474e89094c44da98b954eedeac495271d0f"  # DAI on Ethereum Mainnet
WETH_ADDRESS = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'  # WETH
USDC_ADDRESS = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC

In [298]:
AAVE_POOL_DATA_PROVIDER_ABI = json.loads('[{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveData","outputs":[{"internalType":"uint256","name":"unbacked","type":"uint256"},{"internalType":"uint256","name":"accruedToTreasuryScaled","type":"uint256"},{"internalType":"uint256","name":"totalAToken","type":"uint256"},{"internalType":"uint256","name":"totalStableDebt","type":"uint256"},{"internalType":"uint256","name":"totalVariableDebt","type":"uint256"},{"internalType":"uint256","name":"liquidityRate","type":"uint256"},{"internalType":"uint256","name":"variableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"stableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"averageStableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"liquidityIndex","type":"uint256"},{"internalType":"uint256","name":"variableBorrowIndex","type":"uint256"},{"internalType":"uint40","name":"lastUpdateTimestamp","type":"uint40"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveTokensAddresses","outputs":[{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtTokenAddress","type":"address"},{"internalType":"address","name":"variableDebtTokenAddress","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getAllReservesTokens","outputs":[{"components":[{"internalType":"string","name":"symbol","type":"string"},{"internalType":"address","name":"tokenAddress","type":"address"}],"internalType":"struct IPoolDataProvider.TokenData[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"}]')


In [299]:
# Funct Sig
getAllReservesTokens_4b='b316ff89'
getReserveData_4b='35ea6a75'
getReserveTokensAddresses_4b='d2493b6c'

In [300]:
# getReserveData(address) method Response ]
#   unbacked   uint256 :  0
#   accruedToTreasuryScaled   uint256 :  23618787780361774928
#   totalAToken   uint256 :  2039801843604753838311618
#   totalStableDebt   uint256 :  0
#   totalVariableDebt   uint256 :  1764755511852628454833269
#   liquidityRate   uint256 :  19086348066645041248157072
#   variableBorrowRate   uint256 :  25954495065727705440586566
#   stableBorrowRate   uint256 :  0
#   averageStableBorrowRate   uint256 :  0
#   liquidityIndex   uint256 :  1040724217363078276332975414
#   variableBorrowIndex   uint256 :  1064692592486700566822429320
#   lastUpdateTimestamp   uint40 :  1740762059

# unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp

# output_types = ['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40']

In [301]:
# web3 instance, function from web3py
w3 = web3.Web3()
a3 = w3.eth.contract(address = AAVE_POOL_DATA_PROVIDER, abi=AAVE_POOL_DATA_PROVIDER_ABI)
m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

In [302]:
tokens = [
    '0x6B175474E89094C44Da98b954EedeAC495271d0F',  # DAI
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',  # WETH
    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC
]

In [303]:
calls = []
for token in tokens:
    # Encode the call to getReserveData for this token
    calldata = a3.encode_abi("getReserveData", args=[token])
    calls.append([AAVE_POOL_DATA_PROVIDER_ADDRESS, calldata])

In [304]:
calls

[['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a750000000000000000000000006b175474e89094c44da98b954eedeac495271d0f'],
 ['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a75000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'],
 ['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a75000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48']]

In [305]:
# result = m3.functions.tryAggregate(False, calls).call()
calldata_aave_pool = m3.encode_abi("tryAggregate", args=[False, calls])

In [306]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_aave_pool],
     output_format="polars",
    **cryo_kwargs,
)

In [307]:
calldata = eth_call_df['output_data'][0]

In [308]:
decoded_data = decode(['(bool,bytes)[]'] , calldata)[0]

In [309]:
decoded_data

((True,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x8a\x13\x04^s\xc9\x95}\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8awV\x17\xce\xd5\x18\xfcAOV\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\x93#eW3/ \x89\xdf,\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00#\xd7,\xb1\xc9\xc2\x16\x89\x99\xcf\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00?\xe3\x87J\xf4,\\5c\xf6\x11\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [311]:
token_data = {
    'DAI_RESERVE_DATA': [],
    'WETH_RESERVE_DATA': [],
    'USDC_RESERVE_DATA': []
}

#### DAI Pools Data

In [312]:
dai_reserve_data = decoded_data[0]

In [313]:
data_dai=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])

In [314]:
token_data['DAI_RESERVE_DATA'] = list(data_dai)

In [315]:
token_data

{'DAI_RESERVE_DATA': [0,
  7269387489079587077631,
  167395312853813464274128726,
  0,
  125214200231692820982390572,
  43328536948052255141646305,
  77236799538773625927890449,
  0,
  0,
  1118541717815239899901988787,
  1172943436226722041946359080,
  1740776267],
 'WETH_RESERVE_DATA': [],
 'USDC_RESERVE_DATA': []}

In [244]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])

####  WETH Pools Data

In [316]:
weth_reserve_data = decoded_data[1]

In [317]:
data_weth=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])

In [318]:
token_data['WETH_RESERVE_DATA'] = list(data_weth)

In [255]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])

In [319]:
token_data

{'DAI_RESERVE_DATA': [0,
  7269387489079587077631,
  167395312853813464274128726,
  0,
  125214200231692820982390572,
  43328536948052255141646305,
  77236799538773625927890449,
  0,
  0,
  1118541717815239899901988787,
  1172943436226722041946359080,
  1740776267],
 'WETH_RESERVE_DATA': [0,
  26831704126852729163,
  2052161981910749632571760,
  0,
  1764663515613320616762176,
  18855102222827216705213313,
  25796786545998013490783270,
  0,
  0,
  1040734174170831724081696669,
  1064706039539121825859486629,
  1740777503],
 'USDC_RESERVE_DATA': []}

####  USDC Pools Data

In [320]:
usdc_reserve_data = decoded_data[2]

In [321]:
data_usdc=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])

In [322]:
token_data['USDC_RESERVE_DATA'] = list(data_usdc)

In [286]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])

In [352]:
token_data

{'DAI_RESERVE_DATA': [0,
  7269387489079587077631,
  167395312853813464274128726,
  0,
  125214200231692820982390572,
  43328536948052255141646305,
  77236799538773625927890449,
  0,
  0,
  1118541717815239899901988787,
  1172943436226722041946359080,
  1740776267],
 'WETH_RESERVE_DATA': [0,
  26831704126852729163,
  2052161981910749632571760,
  0,
  1764663515613320616762176,
  18855102222827216705213313,
  25796786545998013490783270,
  0,
  0,
  1040734174170831724081696669,
  1064706039539121825859486629,
  1740777503],
 'USDC_RESERVE_DATA': [0,
  49848480372,
  3006394758211805,
  0,
  1832512246147628,
  34527396073985055763019480,
  62940262213182550579267046,
  0,
  0,
  1122135633059272475406649661,
  1159587027782280670643445974,
  1740777479]}

### Aave Pool Data

In [391]:
field_names = [
    'unbacked', 'accruedToTreasuryScaled', 'totalAToken', 'totalStableDebt', 
    'totalVariableDebt', 'liquidityRate', 'variableBorrowRate', 'stableBorrowRate', 
    'averageStableBorrowRate', 'liquidityIndex', 'variableBorrowIndex', 'lastUpdateTimestamp'
]

In [392]:
df = pd.DataFrame(token_data, index=field_names)

In [393]:
df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,7269387489079587077631,26831704126852729163,49848480372
totalAToken,167395312853813464274128726,2052161981910749632571760,3006394758211805
totalStableDebt,0,0,0
totalVariableDebt,125214200231692820982390572,1764663515613320616762176,1832512246147628
liquidityRate,43328536948052255141646305,18855102222827216705213313,34527396073985055763019480
variableBorrowRate,77236799538773625927890449,25796786545998013490783270,62940262213182550579267046
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1118541717815239899901988787,1040734174170831724081696669,1122135633059272475406649661


In [394]:
readable_df = df

In [395]:
readable_df.loc['lastUpdateTimestamp'] = readable_df.loc['lastUpdateTimestamp'].apply(
    lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')
)

In [396]:
readable_df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,7269387489079587077631,26831704126852729163,49848480372
totalAToken,167395312853813464274128726,2052161981910749632571760,3006394758211805
totalStableDebt,0,0,0
totalVariableDebt,125214200231692820982390572,1764663515613320616762176,1832512246147628
liquidityRate,43328536948052255141646305,18855102222827216705213313,34527396073985055763019480
variableBorrowRate,77236799538773625927890449,25796786545998013490783270,62940262213182550579267046
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1118541717815239899901988787,1040734174170831724081696669,1122135633059272475406649661


In [397]:
readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] = readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] = readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] / 1e18

In [398]:
readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] = readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] / 1e6
readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] / 1e6

In [399]:
readable_df.loc['totalAToken'] = readable_df.loc['totalAToken'].apply(lambda x: f"{float(x):,.4f}" if isinstance(x, (int, float, str)) else x)
readable_df.loc['totalVariableDebt'] = readable_df.loc['totalVariableDebt'].apply(lambda x: f"{float(x):,.4f}" if isinstance(x, (int, float, str)) else x)


In [400]:
# Convert rates from ray (10^27) to percentage for all tokens
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'] / 1e27 * 100
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'] / 1e27 * 100

In [401]:
# Format the rate fields with 4 decimal places and add % sign
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'].apply(lambda x: f"{float(x):.4f}%" if isinstance(x, (int, float, str)) else x)
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'].apply(lambda x: f"{float(x):.4f}%" if isinstance(x, (int, float, str)) else x)

In [402]:
# readable_df.loc['totalAToken'] = df.loc['totalAToken'].apply(lambda x: f"{x:,.4f}")
# readable_df.loc['totalVariableDebt'] = df.loc['totalVariableDebt'].apply(lambda x: f"{x:,.4f}")
# readable_df.loc['liquidityRate'] = df.loc['liquidityRate'].apply(lambda x: f"{x:.4f}%")
# readable_df.loc['variableBorrowRate'] = df.loc['variableBorrowRate'].apply(lambda x: f"{x:.4f}%")

In [403]:
readable_df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,7269387489079587077631,26831704126852729163,49848480372
totalAToken,"167,395,312.8538","2,052,161.9819","3,006,394,758.2118"
totalStableDebt,0,0,0
totalVariableDebt,"125,214,200.2317","1,764,663.5156","1,832,512,246.1476"
liquidityRate,4.3329%,1.8855%,3.4527%
variableBorrowRate,7.7237%,2.5797%,6.2940%
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1118541717815239899901988787,1040734174170831724081696669,1122135633059272475406649661
